In [1]:
import agate

table = agate.Table.from_csv('indivs_Washington16.csv')

In [8]:
pivot = table.pivot(['Contrib', 'Orgname', 'Zip'], aggregation=agate.Sum('Amount')).order_by('Sum', reverse=True)

[u'Contrib', u'Orgname', u'Zip', u'Sum']

In [10]:
print [c.name for c in table.columns]
for row in pivot.order_by('Sum', reverse=True).limit(10).rows:
    print row['Contrib'], row['Orgname'], row['Zip'], row['Sum']

[u'Cycle', u'Fectransid', u'Contribid', u'Contrib', u'Recipid', u'Orgname', u'Ultorg', u'Realcode', u'Date', u'Amount', u'Street', u'City', u'State', u'Zip', u'Recipcode', u'Type', u'CmteId', u'OtherID', u'Gender', u'Microfilm', u'Occupation', u'Employer', u'Source']
BALLMER, CONNIE E Partners for Our Children 98004 250000
PUYALLUP TRIBE OF INDIANS Puyallup Tribe of Indians 98404 186100
SCHWEITZER, BEATRIZ VALDEZ MRS Schweitzer Engineering Labs 99163 173000
SCHWEITZER, EDMUND O DR Schweitzer Engineering Labs 99163 173000
SCHWEITZER, BEATRIZ V Schweitzer Engineering Labs 99163 155000
SCHWEITZER, III, EDMUND O Schweitzer Engineering Labs 99163 150000
None Homemaker 99163 106200
None SCHWEITZER ENGINEERING LABS 99163 106200
SNOQUALMIE TRIBE Snoqualmie Tribe 98065 104800
MUCKLESHOOT INDIAN TRIBE Muckleshoot Indian Tribe 98092 75700


In [11]:
import json
import os
import pycurl
from StringIO import StringIO

def get_cmte_from_id(cmte_id):
    api_key = os.getenv('PROPUBLICA_API_KEY')
    url = 'https://api.propublica.org/campaign-finance/v1/2016/committees/%s.json' % cmte_id

    buffer = StringIO()
    c = pycurl.Curl()
    c.setopt(c.URL, url)
    c.setopt(c.HTTPHEADER, ['X-API-KEY: %s' % api_key])
    c.setopt(c.WRITEDATA, buffer)
    c.perform()
    c.close()

    body = buffer.getvalue()
    # Body is a string in some encoding.
    # In Python 2, we can print it without knowing what the encoding is.
    try:
        return json.loads(body)['results'][0]
    except:
        return {}

In [12]:
mapping = {}

for cmte in table.columns['CmteId'].values_distinct():
    mapping[cmte] = get_cmte_from_id(cmte)

print len(mapping)

776


In [13]:
with open('mapping.json', 'w+') as fh:
    fh.write(json.dumps(mapping))

In [5]:
with open('mapping.json') as fh:
    mapping = json.loads(fh.read())

In [14]:
with_cmtes = table.compute([
    ('CmteName', agate.Formula(agate.Text(), lambda k: mapping[k['CmteId']]['name'])),
    ('CmteParty', agate.Formula(agate.Text(), lambda k: mapping[k['CmteId']]['party'])),
    ('CmteAddress', agate.Formula(agate.Text(), lambda k: mapping[k['CmteId']]['address'])),
    ('CmteCity', agate.Formula(agate.Text(), lambda k: mapping[k['CmteId']]['city'])),
    ('CmteState', agate.Formula(agate.Text(), lambda k: mapping[k['CmteId']]['state'])),
    ('CmteZip', agate.Formula(agate.Text(), lambda k: mapping[k['CmteId']]['zip'])),
    ('CmteType', agate.Formula(agate.Text(), lambda k: mapping[k['CmteId']]['committee_type'])),
])

In [15]:
with_cmtes.to_csv('contribs_with_cmtes.csv')

In [24]:
top_cmtes_pivot = with_cmtes.pivot(['CmteName'], aggregation=agate.Sum('Amount')).order_by('Sum', reverse=True)

In [23]:
for row in top_cmtes_pivot.order_by('Sum', reverse=True).limit(10).rows:
    print '%s raised %d' % (row['CmteName'], row['Sum'])

PEOPLE FOR PATTY MURRAY raised 983429
HILLARY FOR AMERICA raised 860582
NRCC raised 763246
DSCC raised 572685
MURRAY VICTORY FUND raised 498006
ACTBLUE raised 414294
REPUBLICAN NATIONAL COMMITTEE raised 385715
BERNIE 2016 raised 384694
PEOPLE FOR DEREK KILMER raised 354832
WASHINGTON STATE REPUBLICAN PARTY raised 340495


abrahamepton
